# Atividade 4 - Seleção de Características


Entrega: 17/04/2020 às 08:30

ATIVIDADE: Seleção de características usando dois métodos à escolha (de preferência o mais distintos possível)

Métodos: uso de quais rotinas, parâmetros utilizados e
por quê, outras informações relevantes
 Por ex: no caso do Relief, como selecionou as
características com base nos pesos e por quê

Resultados: quantas e quais características foram
selecionadas

Discussão: o que você achou dos resultados? Reduziu
bastante? 

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn import preprocessing

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

## Import do Dataset

In [2]:
df = pd.read_csv('../data/kag_risk_factors_cervical_cancer.csv')
print('-----------------')
print('Tamanho da Matriz')
print(df.shape)
print('-----------------')

-----------------
Tamanho da Matriz
(858, 36)
-----------------


In [3]:
df.head(10)

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0
5,42,3.0,23.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
6,51,3.0,17.0,6.0,1.0,34.0,3.4,0.0,0.0,1.0,...,?,?,0,0,0,0,1,1,0,1
7,26,1.0,26.0,3.0,0.0,0.0,0.0,1.0,2.0,1.0,...,?,?,0,0,0,0,0,0,0,0
8,45,1.0,20.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,1,0,1,1,0,0,0,0
9,44,3.0,15.0,?,1.0,1.266972909,2.8,0.0,0.0,?,...,?,?,0,0,0,0,0,0,0,0


## Pré-Processamento

In [4]:
df.replace('?', np.nan, inplace = True)

In [5]:
df_processed = df.apply(pd.to_numeric, errors='ignore')

In [6]:
df_processed.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
2,34,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,NaN,NaN,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0


## Estastísticas do Dataset

In [7]:
df_processed.describe()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
count,858.000000,832.000000,851.000000,802.000000,845.000000,845.000000,845.000000,750.000000,750.000000,741.000000,...,71.000000,71.000000,858.000000,858.000000,858.000000,858.000000,858.000000,858.000000,858.000000,858.000000
mean,26.820513,2.527644,16.995300,2.275561,0.145562,1.219721,0.453144,0.641333,2.256419,0.112011,...,6.140845,5.816901,0.020979,0.010490,0.020979,0.027972,0.040793,0.086247,0.051282,0.064103
std,8.497948,1.667760,2.803355,1.447414,0.352876,4.089017,2.226610,0.479929,3.764254,0.315593,...,5.895024,5.755271,0.143398,0.101939,0.143398,0.164989,0.197925,0.280892,0.220701,0.245078
min,13.000000,1.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,2.000000,15.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25.000000,2.000000,17.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,...,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,32.000000,3.000000,18.000000,3.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,...,8.000000,7.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,84.000000,28.000000,32.000000,11.000000,1.000000,37.000000,37.000000,1.000000,30.000000,1.000000,...,22.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df_processed.isnull().sum()

Age                                     0
Number of sexual partners              26
First sexual intercourse                7
Num of pregnancies                     56
Smokes                                 13
Smokes (years)                         13
Smokes (packs/year)                    13
Hormonal Contraceptives               108
Hormonal Contraceptives (years)       108
IUD                                   117
IUD (years)                           117
STDs                                  105
STDs (number)                         105
STDs:condylomatosis                   105
STDs:cervical condylomatosis          105
STDs:vaginal condylomatosis           105
STDs:vulvo-perineal condylomatosis    105
STDs:syphilis                         105
STDs:pelvic inflammatory disease      105
STDs:genital herpes                   105
STDs:molluscum contagiosum            105
STDs:AIDS                             105
STDs:HIV                              105
STDs:Hepatitis B                  

## Tratando valores faltantes

In [9]:
#preenchendo com a mediana
imp_median = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp_median = imp_median.fit(df_processed[['Number of sexual partners', 
                                    'First sexual intercourse',
                                    'Num of pregnancies',
                                    'Smokes (years)',
                                    'Smokes (packs/year)',
                                    'Hormonal Contraceptives (years)',
                                    'IUD (years)', 
                                    'STDs (number)',
                                    'STDs: Time since first diagnosis',
                                    'STDs: Time since last diagnosis']])

In [10]:
#preenchendo com o valor mais frequente
imp_most_freq = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imp_most_freq = imp_most_freq.fit(df_processed[['Smokes',
                                                'Hormonal Contraceptives',
                                                'IUD',
                                                'STDs',
                                                'STDs:condylomatosis',
                                                'STDs:cervical condylomatosis',
                                                'STDs:vaginal condylomatosis',
                                                'STDs:vulvo-perineal condylomatosis',
                                                'STDs:syphilis',
                                                'STDs:pelvic inflammatory disease',
                                                'STDs:genital herpes',
                                                'STDs:molluscum contagiosum',
                                                'STDs:AIDS',
                                                'STDs:HIV',
                                                'STDs:Hepatitis B',
                                                'STDs:HPV']])

In [11]:
#atribuindo a mediana para os valores faltantes
df_processed[['Number of sexual partners', 
              'First sexual intercourse',
              'Num of pregnancies',
              'Smokes (years)',
              'Smokes (packs/year)',
              'Hormonal Contraceptives (years)',
              'IUD (years)', 
              'STDs (number)',
              'STDs: Time since first diagnosis',
              'STDs: Time since last diagnosis']] = imp_median.transform(
                                                df_processed[['Number of sexual partners', 
                                                              'First sexual intercourse',
                                                              'Num of pregnancies',
                                                              'Smokes (years)',
                                                              'Smokes (packs/year)',
                                                              'Hormonal Contraceptives (years)',
                                                              'IUD (years)', 
                                                              'STDs (number)',
                                                              'STDs: Time since first diagnosis',
                                                              'STDs: Time since last diagnosis']])

In [12]:
#atribuindo os valores mais frequentes para os valores faltantes
df_processed[['Smokes',
            'Hormonal Contraceptives',
            'IUD',
            'STDs',
            'STDs:condylomatosis',
            'STDs:cervical condylomatosis',
            'STDs:vaginal condylomatosis',
            'STDs:vulvo-perineal condylomatosis',
            'STDs:syphilis',
            'STDs:pelvic inflammatory disease',
            'STDs:genital herpes',
            'STDs:molluscum contagiosum',
            'STDs:AIDS',
            'STDs:HIV',
            'STDs:Hepatitis B',
            'STDs:HPV']] = imp_most_freq.transform(
                                                df_processed[['Smokes',
                                                              'Hormonal Contraceptives',
                                                              'IUD',
                                                              'STDs',
                                                              'STDs:condylomatosis',
                                                              'STDs:cervical condylomatosis',
                                                              'STDs:vaginal condylomatosis',
                                                              'STDs:vulvo-perineal condylomatosis',
                                                              'STDs:syphilis',
                                                              'STDs:pelvic inflammatory disease',
                                                              'STDs:genital herpes',
                                                              'STDs:molluscum contagiosum',
                                                              'STDs:AIDS',
                                                              'STDs:HIV',
                                                              'STDs:Hepatitis B',
                                                              'STDs:HPV']])

## Normalizacao

In [13]:
# Aplica MinMax no grupo de treino
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))

df_normalized = minmax_scale.fit_transform(df_processed)

In [14]:
# transforma de volta para dataframe
df_normalized = pd.DataFrame(df_normalized, columns = df_processed.columns.tolist())

df_normalized.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,0.070423,0.111111,0.227273,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,...,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.028169,0.000000,0.181818,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,...,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.295775,0.000000,0.318182,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,...,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.549296,0.148148,0.272727,0.363636,1.0,1.0,1.0,1.0,0.1,0.0,...,0.142857,0.095238,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.464789,0.074074,0.500000,0.363636,0.0,0.0,0.0,1.0,0.5,0.0,...,0.142857,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def recursiveFeatureElimination(df_norm, qtdFeatures):
    array_values = df_norm.values
    size = len(df_norm.columns) - 1

    X_norm = array_values[:,0:size]
    Y_norm = array_values[:,size]

    model = LogisticRegression(solver='lbfgs')
    rfe = RFE(model, qtdFeatures)
    fit = rfe.fit(X_norm, Y_norm)

    bestFeatures = []
    
    for i in range(len(fit.support_)):
        if fit.support_[i] == True:
            bestFeatures.append(df_norm.columns[i])
        
    #print("Num Features: %d" % fit.n_features_)
    #print("Selected Features: %s" % fit.support_)
    #print("Feature Ranking: %s" % fit.ranking_)

    return bestFeatures

In [16]:
def stepForwardFeatureSelection(df_norm, qtdFeatures):
    X_train, X_test, y_train, y_test = train_test_split(
    df_norm.values[:,:-1],
    df_norm.values[:,-1:],
    test_size=0.25,
    random_state=42)
    
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    
    #print('Training dataset shape:', X_train.shape, y_train.shape)
    #print('Testing dataset shape:', X_test.shape, y_test.shape)
    
    # Build RF classifier to use in feature selection
    clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    # Build step forward feature selection
    sfs1 = sfs(clf,
           k_features=qtdFeatures,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

    # Perform SFFS
    sfs1 = sfs1.fit(X_train, y_train)
    
    # Which features?
    bestFeatures = []
    feat_cols = list(sfs1.k_feature_idx_)
    #print(feat_cols)
    
    for i in range(len(feat_cols)):
        bestFeatures.append(df_norm.columns[feat_cols[i]])
        
    return bestFeatures            

In [17]:
bestFeaturesRFE  = recursiveFeatureElimination(df_normalized, 20)
bestFeaturesSFFS = stepForwardFeatureSelection(df_normalized, 20)
 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   47.9s finished

[2020-04-12 16:03:15] Features: 1/20 -- score: 0.9626816860465116[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   45.6s finished

[2020-04-12 16:04:01] Features: 2/20 -- score: 0.9642320736434108[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:   46.6s finished

[2020-04-12 16:04:48] Features: 3/20 -- score: 0.9657945736434108[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

-------------------
Características RFE
['Number of sexual partners', 'Num of pregnancies', 'Smokes (packs/year)', 'Hormonal Contraceptives (years)', 'STDs', 'STDs:condylomatosis', 'STDs:syphilis', 'STDs:genital herpes', 'STDs:HIV', 'STDs:HPV', 'STDs: Number of diagnosis', 'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis', 'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller', 'Citology']
--------------------
Características SFFS
['Age', 'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Hormonal Contraceptives', 'IUD (years)', 'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis', 'STDs:syphilis', 'STDs:pelvic inflammatory disease', 'STDs:genital herpes', 'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Time since first diagnosis', 'Dx:Cancer', 'Dx:HPV', 'Dx', 'Schiller']


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:   22.5s finished

[2020-04-12 16:14:48] Features: 20/20 -- score: 0.9611555232558139

In [24]:
print('-----------------------------------------------------------------------------------------------------')    
print('Características RFE')
print('\n')
print(bestFeaturesRFE)
print('\n')
print('-----------------------------------------------------------------------------------------------------')
print('\n')
print('Características SFFS')
print('\n')
print(bestFeaturesSFFS)
print('-----------------------------------------------------------------------------------------------------')

-----------------------------------------------------------------------------------------------------
Características RFE


['Number of sexual partners', 'Num of pregnancies', 'Smokes (packs/year)', 'Hormonal Contraceptives (years)', 'STDs', 'STDs:condylomatosis', 'STDs:syphilis', 'STDs:genital herpes', 'STDs:HIV', 'STDs:HPV', 'STDs: Number of diagnosis', 'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis', 'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller', 'Citology']


-----------------------------------------------------------------------------------------------------


Características SFFS


['Age', 'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Hormonal Contraceptives', 'IUD (years)', 'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis', 'STDs:syphilis', 'STDs:pelvic inflammatory disease', 'STDs:genital herpes', 'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Time since first diagnosis', 'Dx:Cancer', 'D